In [37]:
import pandas as pd
import numpy as np
import datetime

from util.OddsCalculator import OddsCalculator as util

In [38]:
#Raw odds from two sources
# 1. https://www.sportsbettingdime.com/politics/2020-us-presidential-election-odds/
# 2. https://www.gamblingsites.org/blog/2020-election-betting-odds-election-night-changes/

odds = pd.read_excel('data/Election Odds Raw.xlsx').set_index('date')

In [39]:
odds

,trump_odds,biden_odds,source
date,,,
2020-04-08 00:00:00,-121,114,1
2020-04-09 00:00:00,-121,114,1
2020-04-10 00:00:00,-121,114,1
2020-04-11 00:00:00,-121,114,1
2020-04-12 00:00:00,-121,114,1
...,...,...,...
2020-11-04 17:00:00,490,-780,1
2020-11-05 00:25:00,528,-867,1
2020-11-05 10:20:00,476,-716,1


In [40]:
odds['trump_odds'] = odds['trump_odds'].astype(str)
odds['biden_odds'] = odds['biden_odds'].astype(str)

In [41]:
#adds a plus sign to the American odds. Essential for conversion to probabilities

def add_plus(val):
    if '-' not in val and '+' not in val:
        val = '+' + val
    return val

In [42]:
odds['trump_odds'] = odds['trump_odds'].apply(add_plus)
odds['biden_odds'] = odds['biden_odds'].apply(add_plus)

In [43]:
odds['trump_odds_dec'] = odds['trump_odds'].apply(util.calculate_odds).str[1]
odds['biden_odds_dec'] = odds['biden_odds'].apply(util.calculate_odds).str[1]

In [44]:
#Calculate the book's hold, for fun

odds['hold'] = odds.apply(lambda x: util.calculate_hold([x['trump_odds_dec'], x['biden_odds_dec']]),axis=1)

In [45]:
#Calculate the true odds of each candidate winning

odds['trump_win_perc'] = odds.apply(lambda x: util.actual_probability([x['trump_odds'], x['biden_odds_dec']]),axis=1).str[0]
odds['biden_win_perc'] = odds.apply(lambda x: util.actual_probability([x['trump_odds'], x['biden_odds_dec']]),axis=1).str[1]

In [46]:
#Append a final row indicating Biden's win

final_date = pd.Timestamp(2020,11,7,11,25)
final_trump_win_perc = 0.0
final_biden_win_perc = 1.0

d = {'date': [final_date], 'trump_win_perc': [final_trump_win_perc], 'biden_win_perc': [final_biden_win_perc]}
final_row = pd.DataFrame(data=d).set_index('date')

In [47]:
final_row

,trump_win_perc,biden_win_perc
date,,
2020-11-07 11:25:00,0.0,1.0


In [48]:
odds = odds.append(final_row, sort=False)

In [49]:
odds.to_csv('data/final_odds.csv')

In [50]:
odds

,trump_odds,biden_odds,source,trump_odds_dec,biden_odds_dec,hold,trump_win_perc,biden_win_perc
date,,,,,,,,
2020-04-08 00:00:00,-121,+114,1.0,1.8264462809917354,2.1399999999999997,1.458520,0.5395,0.4605
2020-04-09 00:00:00,-121,+114,1.0,1.8264462809917354,2.1399999999999997,1.458520,0.5395,0.4605
2020-04-10 00:00:00,-121,+114,1.0,1.8264462809917354,2.1399999999999997,1.458520,0.5395,0.4605
2020-04-11 00:00:00,-121,+114,1.0,1.8264462809917354,2.1399999999999997,1.458520,0.5395,0.4605
2020-04-12 00:00:00,-121,+114,1.0,1.8264462809917354,2.1399999999999997,1.458520,0.5395,0.4605
...,...,...,...,...,...,...,...,...
2020-11-05 00:25:00,+528,-867,1.0,6.28,1.1153402537485582,5.287165,0.1508,0.8492
2020-11-05 10:20:00,+476,-716,1.0,5.76,1.1396648044692737,4.858144,0.1652,0.8348
2020-11-05 15:40:00,+675,-1340,1.0,7.75,1.0746268656716418,5.623682,0.1217,0.8783
